In [92]:
# coding: utf-8

# This is the implementation of [**Autoencoder**](http://www.iro.umontreal.ca/~lisa/bib/pub_subject/language/pointeurs/bengio+lecun-chapter2007.pdf)

# **Outline:**
# - Build model and loss function
# - Train model
# - Observe valdidate
# - Test

# **To do:**
# - Hyperparameter tuning
#     + lr
#     + layer1, layer2
#     + betas

# **Modification**
# - Weight initialization with xavier uniform
# - Adam optimization
# - LR decay


import argparse
import os
import pickle
import sys
import time

import numpy
import pandas as pd
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler
import torch.utils.data
from torch.autograd import Variable
from torchvision import transforms

# from utils.models import AE, AE_tanh, AE_dropout
# from utils.utils import getDfWithTime, convert2df, getSubmission, fixTime, evaluation, getnanindex

from models import *

# Define parser
# name = 'bpi_2012'
# name = 'bpi_2013'
name = 'small_log'
#name = 'large_log'

parser = {
    'train': True,
    'test': True,
    'model_class': 'AE',  # AE/AE_tanh
    'model_name': '',
    'data_dir': '../data/',
    'data_file': name + '.csv',
    'nan_pct': 0.3,
    'input_dir': '../input/{}/'.format(name),
    'batch_size': 16,
    'epochs': 10,
    'no_cuda': False,
    'seed': 7,
    'layer1': 50,
    'layer2': 20,
    'lr': 0.001,
    'betas': (0.9, 0.999),
    'lr_decay': 0.99,
}

args = argparse.Namespace(**parser)
args.output_dir = './output/{0}_{1}_{2}/'.format(name, args.nan_pct, args.model_class)

if not os.path.isdir(args.output_dir):
    os.makedirs(args.output_dir)

args.cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

kwargs = {'num_workers': 2, 'pin_memory': True} if args.cuda else {}

In [116]:
import pandas as pd
import numpy as np
import math
from math import sqrt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import accuracy_score, log_loss
import torch
import torch.nn as nn
from datetime import timedelta

'''
Note: 2 way to index an element in pandas df
By Index:
- df.Time[row]
- df.loc[row, 'Time']

By location:
- df.iloc[1, 1]: iloc only take interger

'''
def calculateTimeInterval(missing_data):
    df = missing_data.copy()
    df['TimeInterval'] = (df['CompleteTimestamp'] - df['CompleteTimestamp'].shift(1))
    df.loc[0, 'TimeInterval'] = 0
    return df

def calculateDuration(missing_data):
    df = missing_data.copy()
    df['Duration'] = (df['CompleteTimestamp'] - df['CompleteTimestamp'].iloc[0])
    df['Duration'].iloc[0] = 0
    return df

def calculateCumTimeInterval(missing_data):
    df = missing_data.copy()
    df['CumTimeInterval'] = (df['CompleteTimestamp'] - df['CompleteTimestamp'].iloc[0])
    return df

def calculateCaseTimeInterval(missing_data):
    df = missing_data.copy()
    df['CaseTimeInterval'] = np.nan

    current_point = {}
    current_point[df.loc[0, 'CaseID']] = df.loc[0, 'CompleteTimestamp']

    for i in range(1, df.shape[0]):
        if df.loc[i, 'CaseID'] != df.loc[i-1, 'CaseID']:
            df.loc[i, 'CaseTimeInterval'] = (df.loc[i, 'CompleteTimestamp'] - current_point[df.loc[i, 'CaseID']-1]).total_seconds()
            current_point[df.loc[i, 'CaseID']] = df.loc[i, 'CompleteTimestamp']
            
    return df

def convert2seconds(x):
    x = x.total_seconds()
    return x


def minmaxScaler(caseid, df_case, missing_df_case):
    epsilon = 0.1
    missing_case_storage = {}
    missing_case_storage[caseid] = {}
    
    temp = df_case.copy()
    missing_temp = missing_df_case.copy()
    
    temp['NormalizedTime'] = temp['CumTimeInterval'].copy()
    missing_temp['NormalizedTime'] = missing_temp['CumTimeInterval'].copy()
    
    min_val = temp['CumTimeInterval'].min()
    max_val = temp['CumTimeInterval'].max()
    
    missing_min_val = missing_temp['CumTimeInterval'].min()
    missing_max_val = missing_temp['CumTimeInterval'].max()
    missing_case_storage[caseid]['missing_min_val'] = missing_min_val
    missing_case_storage[caseid]['missing_max_val'] = missing_max_val
    
    for row in range(temp.shape[0]):
        #scale complete df
        temp.iloc[row, temp.columns.get_loc('NormalizedTime')] = (temp.iloc[row, temp.columns.get_loc('CumTimeInterval')] - min_val)/(max_val-min_val+epsilon)
        
        #scale missing df
        missing_temp.iloc[row, missing_temp.columns.get_loc('NormalizedTime')] = (missing_temp.iloc[row, missing_temp.columns.get_loc('CumTimeInterval')] - missing_min_val)/(missing_max_val-missing_min_val+epsilon)  
    return temp, missing_temp, missing_case_storage


def OHE(df, categorical_variables):
    for i in categorical_variables:
        enc_df = pd.get_dummies(df, columns=categorical_variables, drop_first=False)
    return enc_df

def findLongestLength(groupByCase):
    '''This function returns the length of longest case'''
    #groupByCase = data.groupby(['CaseID'])
    maxlen = 1
    for case, group in groupByCase:
        temp_len = group.shape[0]
        if temp_len > maxlen:
            maxlen = temp_len
    return maxlen

def padwithzeros(vector, maxlen):
    '''This function returns the (maxlen, num_features) vector padded with zeros'''
    npad = ((maxlen-vector.shape[0], 0), (0, 0))
    padded_vector = np.pad(vector, pad_width=npad, mode='constant', constant_values=0)
    return padded_vector

def getInput(groupByCase, cols, maxlen):
    full_list = []
    for case, data in groupByCase:
        temp = data.to_numpy()#data.as_matrix(columns=cols)
        temp = padwithzeros(temp, maxlen)
        full_list.append(temp)
    inp = np.array(full_list)
    return inp

def getMeanVar(array, idx=0):
    temp_array = [a[idx] for a in array if not np.isnan(a[idx])]
    mean_val = np.mean(temp_array)
    var_val = np.var(temp_array)
    return mean_val, var_val

def getProbability(recon_test):
    '''This function takes 3d tensor as input and return a 3d tensor which has 
    probabilities for classes of categorical variable'''
    softmax = nn.Softmax()
    #recon_test = recon_test.cpu() #moving data from gpu to cpu for full evaluation
    
    for i in range(recon_test.size(0)):
        cont_values = recon_test[i, :, 0].contiguous().view(recon_test.size(1),1) #(35,1)
        #softmax_values = softmax(recon_test[i, :, 1:])
        softmax_v = softmax()
        softmax_values = softmax_v.fit(recon_test[i, :, 1:])
        if i == 0:
            recon = torch.cat([cont_values, softmax_values], 1)
            recon = recon.contiguous().view(1,recon_test.size(1), recon_test.size(2)) #(1, 35, 8)
        else:
            current_recon = torch.cat([cont_values, softmax_values], 1)
            current_recon = current_recon.contiguous().view(1,recon_test.size(1), recon_test.size(2)) #(1, 35, 8)
            recon = torch.cat([recon, current_recon], 0)
    return recon

def convert2df(predicted_tensor, pad_matrix, cols, test_row_num):
    '''
    This function converts a tensor to a pandas dataframe
    Return: Dataframe with columns (NormalizedTime, PredictedActivity)

    - predicted_tensor: recon
    - df: recon_df_w_normalized_time
    '''
    print("WORKS")
    #predicted_tensor = getProbability(predicted_tensor) #get probability for categorical variables
    predicted_array = predicted_tensor.data.cpu().numpy() #convert to numpy array
    
    #Remove 0-padding
    temp_array = np.array(predicted_array)
    temp_array = temp_array.reshape(predicted_array.shape[0]*predicted_array.shape[1], predicted_array.shape[2])
    temp_array = temp_array[np.any(temp_array != 0, axis=1)]
    
    #check number of row of df
    if temp_array.shape[0] == test_row_num:
        #print('Converting tensor to dataframe...')
        df = pd.DataFrame(temp_array)
        #activity_list = [i for i in cols if i!='NormalizedTime']
        #df['PredictedActivity'] = df[activity_list].idxmax(axis=1) #get label
        #df['PredictedActivity'] = df['PredictedActivity'].apply(lambda x: x.split('_')[1]) #remove prefix
        #df['PredictedActivity'] = df['PredictedActivity'].apply(lambda x: x[9:]) #remove prefix Activity_
        #df = df.drop(activity_list, axis=1)
        #print('Done!!!')
    return df

def inversedMinMaxScaler(caseid, min_max_storage, recon_df_w_normalized_time_case):
    epsilon = 0.1
    
    temp = recon_df_w_normalized_time_case.copy()
    temp['PredictedCumTimeInterval'] = recon_df_w_normalized_time_case['NormalizedTime'].copy()
    
    #should check for nan values here
    #min_val = min_max_storage[caseid]['missing_min_val']
    #max_val = min_max_storage[caseid]['missing_max_val']
    min_val, max_val = findValidMinMax(caseid, min_max_storage)

    for row in range(temp.shape[0]):
        temp.iloc[row, temp.columns.get_loc('PredictedCumTimeInterval')] = min_val + temp.iloc[row, temp.columns.get_loc('NormalizedTime')]*(max_val-min_val+epsilon)
        
    return temp

def findValidMinMax(caseid, min_max_storage):
    min_val_before = 0
    max_val_before= 0
    min_val_after = 0
    max_val_after = 0
    min_val = 0
    max_val = 0
    
    if caseid == len(min_max_storage):
        for i in range(caseid):
            min_val = min_max_storage[caseid-i]['missing_min_val']
            max_val = min_max_storage[caseid-i]['missing_max_val']
            if not np.isnan(min_val) and not np.isnan(max_val):
                break
    else:
        for i in range(caseid):
            min_val_before = min_max_storage[caseid-i]['missing_min_val']
            max_val_before = min_max_storage[caseid-i]['missing_max_val']
            if not np.isnan(min_val_before) and not np.isnan(max_val_before):
                break
    
        for j in range(len(min_max_storage) - caseid+1):
            min_val_after = min_max_storage[caseid+j]['missing_min_val']
            max_val_after = min_max_storage[caseid+j]['missing_max_val']
            if not np.isnan(min_val_after) and not np.isnan(max_val_after):
                break
        min_val = (min_val_before+min_val_after)/2
        max_val = (max_val_before+max_val_after)/2
    return min_val, max_val


def getDfWithTime(recon_df_w_normalized_time, missing_true_test, min_max_storage):
    temp = recon_df_w_normalized_time.copy()
    temp['CaseID'] = missing_true_test['CaseID'].copy()
    recon_groupByCase = temp.groupby(['CaseID'])
    recon_df_w_time = pd.DataFrame(columns=list(temp)+['PredictedCumTimeInterval'])
    
    for caseid, data_case in recon_groupByCase:
        temp_case = inversedMinMaxScaler(caseid, min_max_storage, data_case)
        recon_df_w_time = recon_df_w_time.append(temp_case)
    return recon_df_w_time



def getnanindex(missing_true_df):
    nan_time_index = []
    nan_activity_index = []
    for row in range(missing_true_df.shape[0]):
        if np.isnan(missing_true_df.CumTimeInterval[row]):
            nan_time_index.append(row)

        if not type(missing_true_df.Activity[row]) == str:
            nan_activity_index.append(row)
    return nan_time_index, nan_activity_index

def getSubmission(recon_df_w_time, missing_true_test, complete_true_test, first_timestamp):
    temp = pd.DataFrame(columns=['CaseID', 'TrueActivity', 'PredictedActivity', 'TrueTime', 'PredictedTime'])
    temp['CaseID'] = missing_true_test['CaseID'].copy()
    
    #ground truth
    temp['TrueActivity'] = complete_true_test['Activity'].copy()
    temp['TrueTime'] = complete_true_test['CumTimeInterval'].copy()
    temp['TrueCompleteTimestamp'] = complete_true_test['CompleteTimestamp'].copy()

    #predicted activity
    temp['PredictedActivity'] = missing_true_test['Activity'].copy()
    temp['PredictedTime'] = missing_true_test['CumTimeInterval'].copy()
    temp['PredictedCompleteTimestamp'] = missing_true_test['CompleteTimestamp'].copy()

    for row in range(temp.shape[0]):
        if pd.isnull(temp.loc[row, 'PredictedActivity']):
            temp.loc[row, 'PredictedActivity'] = recon_df_w_time.loc[row, 'PredictedActivity']
        if pd.isnull(temp.loc[row, 'PredictedTime']):
            temp.loc[row, 'PredictedTime'] = recon_df_w_time.loc[row, 'PredictedCumTimeInterval']
            temp.loc[row, 'PredictedCompleteTimestamp'] = first_timestamp+timedelta(seconds=recon_df_w_time.loc[row, 'PredictedCumTimeInterval'])
    return temp

def fixTime(recon_df_w_time):
    groupByCase = recon_df_w_time.groupby(['CaseID'])
    temp = pd.DataFrame(columns=list(recon_df_w_time))

    for caseid, data_case in groupByCase:
        for row in range(1, len(data_case)):
            current = data_case.iloc[row, data_case.columns.get_loc('PredictedTime')]
            previous = data_case.iloc[row-1, data_case.columns.get_loc('PredictedTime')]
            if current < previous:
                data_case.iloc[row, data_case.columns.get_loc('PredictedTime')] = previous
                data_case.iloc[row, data_case.columns.get_loc('PredictedCompleteTimestamp')] = data_case.iloc[row-1, data_case.columns.get_loc('PredictedCompleteTimestamp')]
        temp = temp.append(data_case)
    return temp


def evaluation(submission_df, nan_time_index, nan_activity_index, show=False):
    #eval Time
    true_time = submission_df.loc[nan_time_index, 'TrueTime']
    predicted_time = submission_df.loc[nan_time_index, 'PredictedTime']
    mae_time = mean_absolute_error(true_time, predicted_time)
    rmse_time = sqrt(mean_squared_error(true_time, predicted_time))
    
    #eval Activity
    true_activity = submission_df.loc[nan_activity_index, 'TrueActivity']
    predicted_activity = submission_df.loc[nan_activity_index, 'PredictedActivity']
    acc = accuracy_score(true_activity, predicted_activity)
    
    if show==True: 
        print('Number of missing Time: {}'.format(len(nan_time_index)))
        print('Mean Absolute Error: {:.4f} day(s)'.format(mae_time/86400))
        print('Root Mean Squared Error: {:.4f} day(s)'.format(rmse_time/86400))
        
        print('Number of missing Activity: {}'.format(len(nan_activity_index)))
        print('Accuracy: {:.2f}%'.format(acc*100))
    return mae_time, rmse_time, acc


In [94]:

preprocessed_data_name = os.path.join(args.input_dir, 'preprocessed_data_{}.pkl'.format(args.nan_pct))
with open(preprocessed_data_name, 'rb') as f:
    min_max_storage = pickle.load(f)
    complete_matrix_w_normalized_time_train = pickle.load(f)
    missing_matrix_w_normalized_time_train = pickle.load(f)
    avai_matrix_train = pickle.load(f)
    nan_matrix_train = pickle.load(f)
    complete_matrix_w_normalized_time_val = pickle.load(f)
    missing_matrix_w_normalized_time_val = pickle.load(f)
    avai_matrix_val = pickle.load(f)
    nan_matrix_val = pickle.load(f)
    pad_matrix_val = pickle.load(f)
    complete_matrix_w_normalized_time_test = pickle.load(f)
    missing_matrix_w_normalized_time_test = pickle.load(f)
    avai_matrix_test = pickle.load(f)
    nan_matrix_test = pickle.load(f)
    pad_matrix_test = pickle.load(f)
    cols_w_time = pickle.load(f)
    cols_w_normalized_time = pickle.load(f)


In [95]:
file_name = os.path.join(args.input_dir, 'parameters_{}.pkl'.format(args.nan_pct))
with open(file_name, 'rb') as f:
    most_frequent_activity = pickle.load(f)
    first_timestamp = pickle.load(f)
    avai_instance = pickle.load(f)
    nan_instance = pickle.load(f)
    train_size = pickle.load(f)
    val_size = pickle.load(f)
    test_size = pickle.load(f)
    train_row_num = pickle.load(f)
    val_row_num = pickle.load(f)
    test_row_num = pickle.load(f)



In [96]:
complete_matrix_w_normalized_time_trainLoader = torch.utils.data.DataLoader(complete_matrix_w_normalized_time_train,batch_size=16, shuffle=True, num_workers=2)

complete_matrix_w_normalized_time_trainLoader.transform = transforms.Compose([transforms.ToTensor()])

missing_matrix_w_normalized_time_trainLoader = torch.utils.data.DataLoader(missing_matrix_w_normalized_time_train,
                                                                           batch_size=16, shuffle=True, num_workers=2)
missing_matrix_w_normalized_time_trainLoader.transform = transforms.Compose([transforms.ToTensor()])

avai_matrix_trainLoader = torch.utils.data.DataLoader(avai_matrix_train, batch_size=16, shuffle=True, num_workers=2)
avai_matrix_trainLoader.transform = transforms.Compose([transforms.ToTensor()])

normalized_complete_df_name = os.path.join(args.input_dir, 'normalized_complete_df_{}.csv'.format(args.nan_pct))
normalized_complete_df = pd.read_csv(normalized_complete_df_name)

normalized_missing_df_name = os.path.join(args.input_dir, 'normalized_missing_df_{}.csv'.format(args.nan_pct))
normalized_missing_df = pd.read_csv(normalized_missing_df_name)

missing_true_val = normalized_missing_df[train_row_num:-test_row_num].reset_index(drop=True)
complete_true_val = normalized_complete_df[train_row_num:-test_row_num].reset_index(drop=True)

missing_true_test = normalized_missing_df[-test_row_num:].reset_index(drop=True)
complete_true_test = normalized_complete_df[-test_row_num:].reset_index(drop=True)

missing_true_val.shape,missing_true_test.shape

nan_time_index_val, nan_activity_index_val = getnanindex(missing_true_val)

nan_time_index_test, nan_activity_index_test = getnanindex(missing_true_test)

pd.isnull(normalized_missing_df).sum()

pd.isnull(missing_true_val).sum()

pd.isnull(missing_true_test).sum()

CaseID                  0
Activity             1680
CompleteTimestamp    1701
CumTimeInterval      1701
NormalizedTime       1701
dtype: int64

In [97]:
if args.model_class == 'AE':
    model = AE(complete_matrix_w_normalized_time_train.shape, args.layer1, args.layer2)

if args.model_class == 'AE_tanh':
    model = AE_tanh(complete_matrix_w_normalized_time_train.shape, args.layer1, args.layer2)

if args.model_class == 'AE_dropout':
    model = AE_dropout(complete_matrix_w_normalized_time_train.shape, args.layer1, args.layer2)

if args.cuda:
    model.cuda()

In [98]:
# Define loss
def loss_function(recon_x, x, avai_mask):
    BCE = F.binary_cross_entropy(recon_x, x, weight=avai_mask, size_average=False)
    return BCE


optimizer = optim.Adam(model.parameters(), lr=args.lr, betas=args.betas)

# Adjust learning rate per epoch: http://pytorch.org/docs/master/optim.html?highlight=lr_scheduler#torch.optim.lr_scheduler.LambdaLR

# Method 1:
lambda1 = lambda epoch: args.lr_decay ** epoch
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=[lambda1])



In [99]:
# Method 2:
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10)


# ## Utils


def save_model(model, epoch, score):
    model_file = os.path.join(args.output_dir,
                              'model_{}_epoch{}_score{:.4f}.pth'.format(args.model_class, epoch, score))
    torch.save(model.state_dict(), model_file)


def load_model(model, model_name):
    model_file = os.path.join(args.output_dir, model_name)
    assert os.path.isfile(model_file), 'Error: no model found!'
    model_state = torch.load(model_file)
    model.load_state_dict(model_state)


def val(model, missing_matrix_w_normalized_time_val, complete_matrix_w_normalized_time_val, avai_matrix_val):
    model.eval()
    m_val = missing_matrix_w_normalized_time_val
    #m_val = Variable(torch.Tensor(m_val).float())
    m_val = torch.from_numpy(numpy.array(m_val, dtype='float32'))
    c_val = complete_matrix_w_normalized_time_val
    #c_val = Variable(torch.Tensor(c_val).float())
    c_val = torch.from_numpy(numpy.array(c_val, dtype='float32'))

    #avai_matrix_val = Variable(torch.Tensor(avai_matrix_val).float())
    avai_matrix_val = torch.from_numpy(numpy.array(avai_matrix_val, dtype='float32'))
    if args.cuda:
        m_val = m_val.cuda()
        c_val = c_val.cuda()
        avai_matrix_val = avai_matrix_val.cuda()

    recon_val = model(m_val)
    val_loss = loss_function(recon_val, c_val, avai_matrix_val)

    #return val_loss.data[0] / len(c_val.data)
    return val_loss.data / len(c_val.data)


missing_true_val.head()

complete_true_val.head()



,CaseID,Activity,CompleteTimestamp,CumTimeInterval,NormalizedTime
0,1201,Activity A,1970-01-01 09:00:00,0.0,0.000000
1,1201,Activity B,1970-01-01 10:00:00,3600.0,0.125000
2,1201,Activity C,1970-01-01 11:00:00,7200.0,0.249999
3,1201,Activity D,1970-01-01 12:00:00,10800.0,0.374999
4,1201,Activity E,1970-01-01 13:00:00,14400.0,0.499998


In [100]:
def train(epoch, model, optimizer):
    model.train()
    train_loss = 0

    zips = zip(missing_matrix_w_normalized_time_trainLoader.dataset, complete_matrix_w_normalized_time_trainLoader.dataset, avai_matrix_trainLoader.dataset)
    enums = enumerate(zips)
    for batch_idx, (m_data, c_data, avai_mask)  in enums:

        #c_data = Variable(c_data)
        #m_data = Variable(m_data)
        #avai_mask = Variable(avai_mask)
        c_data = torch.from_numpy(numpy.array(c_data, dtype='float32'))
        m_data = torch.from_numpy(numpy.array(m_data, dtype='float32'))
        avai_mask = torch.from_numpy(numpy.array(avai_mask, dtype='float32'))

        if args.cuda:
            c_data = c_data.cuda()
            m_data = m_data.cuda()
            avai_mask = avai_mask.cuda()

        optimizer.zero_grad()

        recon_data = model(m_data)

        loss = loss_function(recon_data, c_data, avai_mask)

        loss.backward()
        train_loss += loss.data
        optimizer.step()

    return train_loss / len(complete_matrix_w_normalized_time_trainLoader.dataset)



In [101]:

if args.train:
    for epoch in range(1, args.epochs + 1):
        init = time.time()

        # method 1 scheduler
        scheduler.step()
        train_loss = train(epoch, model, optimizer)
        end_train = time.time()

        val_score = val(model, missing_matrix_w_normalized_time_val,
                        complete_matrix_w_normalized_time_val, avai_matrix_val)

        '''
        #save_model(model, epoch, val_score)
        if epoch == 1:
            current_best = val_score
            save_model(model, epoch, val_score)
        
        else:
            if val_score < current_best:
                current_best = val_score
                save_model(model, epoch, val_score)
        '''

        # method 2 scheduler
        # scheduler.step(val_score)

        end = time.time()
        print('====> Epoch {} | Train time: {:.4f} ms| End time: {:.4f} ms | Train loss: {:.4f} | Val score: {:.4f}'.
              format(epoch, (end_train - init) * 1000, (end - init) * 1000, train_loss, val_score))
else:
    load_model(model, args.model_name)


====> Epoch 1 | Train time: 1817.4357 ms| End time: 1820.9398 ms | Train loss: -185612704.0000 | Val score: -763352832.0000
====> Epoch 2 | Train time: 1751.0302 ms| End time: 1755.4929 ms | Train loss: -185680240.0000 | Val score: -763352832.0000
====> Epoch 3 | Train time: 1743.1052 ms| End time: 1747.2475 ms | Train loss: -185680240.0000 | Val score: -763352832.0000
====> Epoch 4 | Train time: 1810.2753 ms| End time: 1813.8976 ms | Train loss: -185680240.0000 | Val score: -763352832.0000
====> Epoch 5 | Train time: 1744.1399 ms| End time: 1747.5216 ms | Train loss: -185680240.0000 | Val score: -763352832.0000
====> Epoch 6 | Train time: 1739.5651 ms| End time: 1742.9273 ms | Train loss: -185680240.0000 | Val score: -763352832.0000
====> Epoch 7 | Train time: 1735.9321 ms| End time: 1739.3064 ms | Train loss: -185680240.0000 | Val score: -763352832.0000
====> Epoch 8 | Train time: 1709.4035 ms| End time: 1712.7128 ms | Train loss: -185680240.0000 | Val score: -763352832.0000
====> Ep

In [102]:

# # Predict and evaluate


# if args.test:

#submission_df.head(10)

#submission.head(10)


In [119]:
    import utils
    m_test = missing_matrix_w_normalized_time_test
    m_test = Variable(torch.Tensor(m_test).float())

    if args.cuda:
        m_test = m_test.cuda()

    print('Predicting...')
    recon_test = model(m_test)
    print('\n')
    
    print('Converting to dataframe...')

    recon_df_w_normalized_time = convert2df(recon_test, pad_matrix_test, cols_w_normalized_time, test_row_num)

    print('Transforming Normalized Time to Time...')
    recon_df_w_time = getDfWithTime(recon_df_w_normalized_time, missing_true_test, min_max_storage)

    print('Getting submission...')
    # submission_df = getSubmission(recon_df_w_time, missing_true_test, complete_true_test, first_timestamp)
    # submission = fixTime(submission_df)

    print('Testing...')
    #mae_time, rmse_time, acc = evaluation(submission, nan_time_index_test, nan_activity_index_test, show=True)
    print('\n')

    print('Saving submission...')
    #submission_df.to_csv(args.output_dir + 'submission.csv', index=False)
    print('Done!')

Predicting...


Converting to dataframe...
WORKS
Transforming Normalized Time to Time...


KeyError: 'NormalizedTime'